## Step by Step of Front Part

#### Imports

In [1]:
from vqa import ImageEmbedding

In [2]:
import cv2
import torch
import numpy as np
import torch.nn as nn

### Image Embedding Class

In [3]:
#From senior's code (detect.py)
image = cv2.imread("images/joTest3.png")
#bg_img = cv2.resize(image, (512, 512))
orig_image = image.copy()
# # BGR to RGB
image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
# make the pixel range between 0 and 1
image /= 255.0
# bring color channels to front
image = np.transpose(image, (2, 0, 1)).astype(np.float64)
# # convert to tensor
image = torch.tensor(image, dtype=torch.float)
# # add batch dimension
image = image.unsqueeze_(0)

In [4]:
embed = ImageEmbedding()

In [6]:
output = embed(image)

## Descriptor Embeddings
sBert for embeddings, need to use the preprocessors to get the descriptors

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
model = SentenceTransformer('sentence-transformers/average_word_embeddings_glove.6B.300d')

#### 10/170489.png File

In [9]:
qa = ["I should buy a Butterfinger because it will make my life more fun. ", "I should eat butterfingers because the simpsoms say", "I should buy this product because it will make me as funny as Bart Simpson."]
sentiment = [["6", "14"], ["12"], ["11"]]
strat = [["5"], ["6"], ["2", "5"], ["2", "6", "8"], ["5", "6", "9"]]
topic = ["2", "2", "2"]

In [10]:
def transform(target_lst):
    """
    Transform the target_lst of sentiments provided by the PITTs dataset to
    a Pytorch tensor based on the Word2Vec model.

    target_list: a list of lists where each element is a number
    """
    # flatten list
    lst = [item for sublist in target_lst for item in sublist]

    # convert to int
    lst = [int(num) for num in lst]

    return max(lst, key=lst.count)

In [56]:
transform(sentiment) #amazed

6

In [57]:
transform(topic) #chocolate

2

In [58]:
transform(strat) #culture

5

In [11]:
sent = torch.tensor(model.encode("amazed"))
stra = torch.tensor(model.encode("culture"))
top = torch.tensor(model.encode("chocolate"))

In [12]:
first = torch.tensor(model.encode(qa[0]))

In [68]:
1024/4

256.0

In [13]:
qa_proc = nn.Sequential(nn.Linear(300,256),nn.Tanh())

In [14]:
qa_small = qa_proc(first)

In [15]:
qa_stra = torch.mul(first, stra)
qa_stra = qa_proc(qa_stra)

In [16]:
qa_top = torch.mul(first, top)
qa_top = qa_proc(qa_top)

In [17]:
qa_sent = torch.mul(first, sent)
qa_sent = qa_proc(qa_sent)

In [18]:
qa_res = torch.cat([qa_sent, qa_small, qa_top, qa_stra], dim=0)

## Mutan Fusion & MLP from VQA
For Lydia to see how the mlp output will be like

In [19]:
from vqa import MutanFusion

In [20]:
mutan = MutanFusion(1024, 1024, 5)
mlp = nn.Sequential(nn.Linear(1024, 50))

In [21]:
combined = mutan(qa_res, output)

c:\Users\Userr\anaconda3\envs\FIT3163Project\lib\site-packages\torch\nn\functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [22]:
final = mlp(combined)